In [1]:
import gzip, json
import random
import re
import pronouncing
import matplotlib.pyplot as plt
import pandas as pd
from textblob import TextBlob
import nltk

In [2]:
#This just makes a list of all lines of poetry, sans extra whitespace
all_lines =[]
for line in gzip.open("gutenberg-poetry-v001.ndjson.gz"):
    all_lines.append(json.loads(line.strip()))

In [3]:
#builds a dictionary that maps words with that rhyming part to the lines of poetry that they're found at the end of.
from collections import defaultdict

by_rhyming_part = defaultdict(lambda: defaultdict(list))

for line in all_lines:
    text = line['s']
    if not(32 < len(text) < 48): # only use lines of uniform lengths
        continue
        # /b indicates word boundaries
        # w+ 'word character' alphanumeric I think
        # W not word character 
        
    match = re.search(r'(\b\w+\b)\W*$', text)
    
    if match:
        last_word = match.group()
        pronunciations = pronouncing.phones_for_word(last_word)
        if len(pronunciations) > 0:
               rhyming_part = pronouncing.rhyming_part(pronunciations[0])
               # group by rhyming phones (for rhymes) and words (to avoid duplicate words)
               by_rhyming_part[rhyming_part][last_word.lower()].append(text)
               
               

In [4]:
lines_frame = pd.DataFrame(by_rhyming_part)

In [5]:
justlines = []
for each in all_lines:
    justlines.append(each['s'])



In [6]:
import string
no_punct = []
for each in justlines:
    punctless = each.strip(string.punctuation)
    no_punct.append(punctless)


### Certain words aren't being caught by my pronunciation checker. Is it 'cause they're old english or something?

In [6]:
#source_word = 'knelling'
#print(source_word.isalpha())
#phones = pronouncing.phones_for_word(source_word)[0] #
#print(phones)
#rhyming_part = pronouncing.rhyming_part(phones)

These words make my prounouncing library dry heave

In [7]:
bad_words =[]
for line in no_punct:
    text = line
    source_word = text.split(' ')[-1]
    try:
        pronouncing.phones_for_word(source_word)[0]
    except IndexError:
        bad_words.append(text.split(' ')[-1])
len(bad_words)    

347149

In [8]:
bad_words = pd.Series(bad_words)
bad_words.sample(5)

321563      scornfully
32336       Eden-glade
237206            woþe
127901         "Father
39112     bridle-reins
dtype: object

That's a huge number of bad words. A lot of them have punctuation at the end. I suspect a lot of those are salvagable.

## Analysis of original corpus

In [9]:
### This doesn't work. But it feels like it should.
def sentimenter(line):
    sentiment= TextBlob(line).sentiment
    subjectivity = sentiment.subjectivity
    return (subjectivity)
def polarityer(line):
    sentiment= TextBlob(line).sentiment
    polarity = sentiment.polarity
    return (polarity)


In [10]:
no_punct_no_repeats = set(no_punct)

In [11]:
corpus = pd.DataFrame(no_punct_no_repeats)
corpus['Line'] = corpus[0]
corpus.drop([0], axis=1, inplace= True)
corpus.head()

,Line
0,From every eye
1,Of the freaked flag and meadow buttercup
2,"So thickly studded, in the depth of Mars"
3,"Fight and fight on, exulting in the light"
4,"Within the dripping churchyard, the rain plash..."


corpus['sentiment'] = corpus['Line'].apply(sentimenter)
print('THE CODE IS RUNNING, I SWEAR.')
corpus['polarity'] = corpus['Line'].apply(polarityer)
corpus.head(2)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [18]:


count_vect = CountVectorizer(ngram_range=(1, 2), stop_words='english')

count_matrix = count_vect.fit_transform(corpus['Line'])

KeyboardInterrupt: 

In [91]:
count_matrix.shape

(2665604, 4551288)

In [92]:

cos_sim = cosine_similarity(count_matrix)

MemoryError: Unable to allocate 391. GiB for an array with shape (52451621499,) and data type int64

In [ ]:
type(cos_sim)

In [12]:
idx_assign = corpus.reset_index() 
def watchme(movie):
    movie_idx = idx_assign[idx_assign['Line'] == movie].index[0]
    sim_array = cos_sim[movie_idx]
    array_lst = list(enumerate(sim_array))
    sort_lst = sorted(array_lst, reverse=True, key = lambda x: x[1])
    top_recs = sort_lst[0:40]
    for i in top_recs:
        print(idx_assign.loc[i[0]].Line)
    return top_recs


In [ ]:
watchme(corpus['Line'].loc[3])

###  The below does not include cosin similarity

This could be resolved by producing a list of similar lines, as above, and then selecting randomly, or the first that rhymes from it

# THE MACHINE

In [273]:
#This whole thing doesn't work super reliably. Some of it has to do with the pronunciation library missing words in my corpus,
#some of it has to do with punctuation. Working on it, but it's not super high priority.
def limirick_machine(tone_param=True):
    #tone_param, sorts sentiment. True = ascending, or sad, false = Descending, or happy
    #Finds you lines that don't need to rhyme
    def initial_line_finder(list_of_existing):
        rand_choice = random.choice(no_punct)
        if rand_choice not in list_of_existing:
            line = rand_choice
            
        while line.split(' ')[-1].isalpha() == False:
            rand_choice = random.choice(no_punct)
            if rand_choice not in list_of_existing:
                line = rand_choice
        
        flag = False
        while flag == False:            
            try:
                
                word_to_rhyme = line.split(' ')[-1]
                phones = pronouncing.phones_for_word(word_to_rhyme)
                phones = phones[0]
                flag= True
                
            except IndexError:  
                
                rand_choice = random.choice(no_punct)
                while line.split(' ')[-1].isalpha() == False:
                    rand_choice = random.choice(no_punct)
                    if rand_choice not in list_of_existing:
                        line = rand_choice
                phones = pronouncing.phones_for_word(word_to_rhyme)
                phones = phones[0]
            
            
            
        
        ###
        
        rhyming_phones = pronouncing.rhyming_part(phones)
        return line, rhyming_phones
    
    # Finds lines that end with similar rhymes.
    def secondary_line_finder(list_of_existing, phones_to_rhyme, cosine_buddy):   
        
        
        rand_choice = ''
        ##############################################
        
        def rhymeme(movie, tone=True):
            def sentimenter(line):
                sentiment= TextBlob(line).sentiment
                subjectivity = sentiment.subjectivity
                return (subjectivity)
                
            movie_idx = idx_assign[idx_assign == movie].index[0]
            idx_assign['line'] = idx_assign[0]
            idx_assign.drop(inplace=True, axis=1, labels=[0])
            sim_array = cos_sim[movie_idx]
            
            
            array_lst = list(enumerate(sim_array))
            

            sort_lst = sorted(array_lst, reverse=True, key = lambda x: x[1])
            cosign_df = pd.DataFrame(sort_lst)
            cosign_df['orig_ind'] = cosign_df[0] 
            
            cosign_df.drop(inplace=True, axis=1, labels=[0])
            cosign_df['cosign_score'] = cosign_df[1]
            cosign_df.drop(inplace=True, axis=1, labels=[1])
            #builds a df with both cosign similarity and sentiment in it
            co_sentiment_df = idx_assign.merge(cosign_df, left_on='index', right_on='orig_ind')
            co_sentiment_df['sentiment'] = co_sentiment_df['line'].apply(sentimenter)
            co_sentiment_df.sort_values(['cosign_score'], ascending=False)
            co_sentiment_df.drop(inplace=True, axis=1, labels=['index', 'orig_ind'])
            #Sorts by cosign
            sorted_cosign = co_sentiment_df.sort_values(['cosign_score'], ascending=False).head(10)
            #Sorts the dataframe by sentiment
            sorted_sentiment = sorted_cosign.sort_values(['sentiment'], ascending=False)
            sorted_sentiment.drop(axis=0, labels=[0], inplace=tone)

            final_phrase = sorted_sentiment['line'].iloc[0]

            return final_phrase
        ##############################################
        
        while rand_choice in list_of_existing or rand_choice =='':
            key = random.choice(list(by_rhyming_part[phones_to_rhyme].keys()))    
            list_of_rhyming_lines = list(by_rhyming_part[phones_to_rhyme][key])

            count_vect = CountVectorizer(ngram_range=(1, 2), stop_words='english')

            to_vectorize = list_of_rhyming_lines
            to_vectorize.append(cosine_buddy)
            
            
            count_matrix = count_vect.fit_transform(to_vectorize)
            cos_sim = cosine_similarity(count_matrix)
            
            rhyming_series = pd.Series(to_vectorize)
            
            

            
            
            
            idx_assign = rhyming_series.reset_index() 
            rand_choice = rhymeme(cosine_buddy, tone_param)
            
            
        return rand_choice
            

    list_of_lines = []
    line1, line1_phones = initial_line_finder(list_of_lines)
    list_of_lines.append(line1)
        
    line2 = secondary_line_finder(list_of_lines, line1_phones, list_of_lines[-1])
    list_of_lines.append(line2)
    
    line3, line3_phones = initial_line_finder(list_of_lines)
    list_of_lines.append(line3)
    
    line4 = secondary_line_finder(list_of_lines, line3_phones,list_of_lines[-1])
    list_of_lines.append(line4)
    
    line5 = secondary_line_finder(list_of_lines, line1_phones,list_of_lines[-1])
    

    return line1, line2, line3, line4, line5

    
    

In [276]:
line1, line2, line3, line4, line5 = limirick_machine(tone_param=True)
print('  ')
print('  ')
print(line1)
print(line2)
print(line3)
print(line4)
print(line5)

  
  
Some indiscreet Abderite boys
"How beautiful the sea! How she enjoys
The lawes of his lond to holde
The sweeping storm, mighty, like flag unrolled
Of dalliance had with thee in Heav'n, and joys


In [53]:
#initial set up.
import pandas as pd
temp_limirick_dict= {'line1':[],'line2':[],'line3':[],'line4':[],'line5':[],}
for i in range(10):
    line1, line2, line3, line4, line5 = limirick_machine()
    temp_limirick_dict['line1'].append(line1)
    temp_limirick_dict['line2'].append(line2)
    temp_limirick_dict['line3'].append(line3)
    temp_limirick_dict['line4'].append(line4)
    temp_limirick_dict['line5'].append(line5)
basic_limirick_df = pd.DataFrame(temp_limirick_dict)


    
    

['Manhood to God-head, with more strength to foil', 'Manhood to Godhead, with more strength to foil', 'Up then and with thy courage put to foil', 'Of Moultrie and of Eutaw--who shall foil', 'THAT shall for ever live and foil', "Of proofs new-bleeding, which remain'd the foil", 'Manhood to God-head, with more strength to foil', 'For vacant glitter.  It were scarce a foil', "And what's more wonderful, when big loads foil", "By no man's arrow reach'd; I fear no foil", 'Hither most gentle sleep! and soothing foil', 'Nor mendicant nor ballad beggar foil', 'Scorning, and bent, with feet of wind, to foil', 'With what gay spirit does she foil', 'Songs of their brother-hours shall foil', 'Or the dull lisp of oaken limbs that foil', 'And even superstition, fails to foil', 'Of Moultrie and of Eutaw--who shall foil', 'I do confess the fairness of the spoil']
['But the great loss should by instinct impair', "Learn his departure, lest thou should'st impair", 'Ulysses, ruthless Chief! no toils impair

IndexError: list index out of range

In [ ]:
#increase size
#I just run this a few times, errors don't hurt if you just run it again.
temp_limirick_dict= {'line1':[],'line2':[],'line3':[],'line4':[],'line5':[],}
for i in range(10):
    line1, line2, line3, line4, line5 = limirick_machine()
    temp_limirick_dict['line1'].append(line1)
    temp_limirick_dict['line2'].append(line2)
    temp_limirick_dict['line3'].append(line3)
    temp_limirick_dict['line4'].append(line4)
    temp_limirick_dict['line5'].append(line5)
temp_limirick_df = pd.DataFrame(temp_limirick_dict)
basic_limirick_df = pd.concat([basic_limirick_df, temp_limirick_df])


In [ ]:
print(basic_limirick_df.shape)
print(basic_limirick_df.sample(1))

In [ ]:
test_value = basic_limirick_df['line1'].iloc[0]
test_value

In [ ]:
### This doesn't work. But it feels like it should.
def sentimenter(line):
    sentiment= TextBlob(line).sentiment
    subjectivity = sentiment.subjectivity
    return (subjectivity)
def polarityer(line):
    sentiment= TextBlob(line).sentiment
    polarity = sentiment.polarity
    return (polarity)

for column in basic_limirick_df:
    basic_limirick_df[column + '_sentiment'] = basic_limirick_df[column].apply(sentimenter)
    basic_limirick_df[column + '_polarity'] = basic_limirick_df[column].apply(polarityer)

In [ ]:
sentimenter1('half lying on the grass, yellow')

In [ ]:
basic_limirick_df.shape

In [ ]:
basic_limirick_df.sample(10)

In [ ]:
line1_negative_polarity_mask = basic_limirick_df.line1_polarity <0
line2_negative_polarity_mask = basic_limirick_df.line2_polarity <0
line3_negative_polarity_mask = basic_limirick_df.line3_polarity <0
line4_negative_polarity_mask = basic_limirick_df.line4_polarity <0
line5_negative_polarity_mask = basic_limirick_df.line5_polarity <0

In [ ]:
#Gotta reset the index when I concatonate or something. But I'm just annotating on this pass.

basic_limirick_df[line1_point2_polarity_mask].head()
print(basic_limirick_df['line1'][line1_negative_polarity_mask][5])
print(basic_limirick_df['line2'][5])
#print(basic_limirick_df['line3'][5])
#print(basic_limirick_df['line4'][5])
#print(basic_limirick_df['line5'][5])

In [ ]:
#make a wordcloud
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["br", "href"])
textt = " ".join(review for review in basic_limirick_df.line2)
wordcloud = WordCloud(stopwords=stopwords).generate(textt)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud11.png')
plt.show()